# Poor man's gps tracking 

In [ ]:
import os

import tqdm

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

<br>

## Load data

### Pregenerated 

In [ ]:
base = """./geo_traces/"""

In [ ]:
track_gps = pd.read_csv(os.path.join(base, "loop_gps_M9.csv"))
track_true = pd.read_csv(os.path.join(base, "loop_true_M9.csv"))

M = 9

<br>

### Custom track generator

<br>

### Visualize

Keep the tracking data in `P`.

In [ ]:
P = np.ascontiguousarray(track_gps.to_numpy())

In [ ]:
from sklearn.cluster import KMeans

cluster = KMeans(n_clusters=M)

aff = cluster.fit_predict(P)

plt.plot(track_true.X, track_true.Y, c="k", zorder=10)
colors = plt.cm.rainbow(np.linspace(0, 1, 14))
plt.scatter(track_gps.X, track_gps.Y, c=colors[aff])

<br>

## DP approach 

### The objective

Consider some tracking data $(x_k)_{k=0}^{K-1} \in \mathbb{R}^d$.
It is known that true position is on a continuous piecewise
linear track and is sampled at a fixed rate with Gaussian
noise $\mathcal{N}_d(0, \sigma^2 I)$ with unknown $\sigma^2$.

Additional info:
* the object moves at constant speed

The true track consists of $M$ line segments:

$$
\mu
    \colon [0, 1] \to \mathbb{R}^d
    \colon \tau \mapsto \sum_{m=1}^M 1_{[t_m, t_{m+1})}(\tau)
    \,\mu_m\Bigl(\frac{\tau - t_m}{t_{m+1} - t_m}\Bigr)
    \,, $$

where $0 = t_1 < \cdots < t_{M+1} = 1$ and the $m$-th segment is $
    \mu_m
    \colon [0, 1] \to \mathbb{R}^d
    \colon \tau \mapsto (1 - \tau) p_m + \tau p_{m+1}
$ for $(p_m)_{m=1}^{M+1} \in \mathbb{R}^d$.

Observations $
    x_k \sim \mathcal{N}_d\bigl(
        \mu(\tau_k), \sigma^2 I_d
    \bigr)
$ indep., $k=0,.., K-1$.

The goal is to minimize the following objective 
$$
\frac{dK}2 \log \sigma^2
    + \frac1{2 \sigma^2} \sum_{k=0}^{K-1}
        \bigl\| x_k - \mu(\tfrac{k}{K}) \bigr\|^2
    = 
    \frac{dK}2 \log \sigma^2
    + \frac1{2 \sigma^2} \sum_{m=1}^M \sum_{k\in [s_m, s_{m+1})}
        \bigl\| x_k - \mu_m\bigl(\tfrac{k - s_m}{s_{m+1} - s_m}\bigr) \bigr\|^2
    \,, $$
with respect to $\sigma^2$, $p = (p_m)_{m=1}^{M+1}$, $t = (t_m)_{m=1}^{M+1}\in [0, 1]$
($t_1=0$, $t_{M+1}=1$) and $s = t K$.

Obviously, given $p_m$ and $s_m$ the optimal $\sigma^2$ is just
$$
\sigma^2
    = \frac1{dK} \sum_{m=1}^M \sum_{k\in [s_m, s_{m+1})}
        \bigl\|
            x_k - \mu_m\bigl(\tfrac{k - s_m}{s_{m+1} - s_m}\bigr)
        \bigr\|^2
    \,.$$

Thus the objective becomes

$$
\frac{dK}2 \log \frac1{dK} \sum_{m=1}^M \sum_{k\in [s_m, s_{m+1})}
        \bigl\|
            x_k - \mu_m\bigl(\tfrac{k - s_m}{s_{m+1} - s_m}\bigr)
        \bigr\|^2
    + \frac{dK}2
    \,, $$

which is equivalent to

$$
\frac1{dK} \sum_{m=1}^M \sum_{k\in [s_m, s_{m+1})}
    \bigl\|
        x_k - \mu_m\bigl(\tfrac{k - s_m}{s_{m+1} - s_m}\bigr)
    \bigr\|^2
    \,, $$

due to monotonicity of $\log$. This should be minimized with respect to splits
$(s_m)_{m=1}^{M+1} \in [0, K]$ and knots $(p_m)_{m=1}^{M+1}$.

This looks suspiciously like a clustering objective.

We assume $p_0 = 0$, i.e. the piecewise track starts at zero and $\mu(0) = p_0 = 0$.

<br>

### DP: Times and points (unfinished)

Let's try dyn-prog appriach here. The data is $(\tau_k, x_k) \in [0, 1] \times \mathbb{R}^d$
and the (inner) objective is

$$
L(p, t; M, K)
    = \sum_{m=1}^M \sum_{k=1}^K [k\in I_m]
    \Bigl\|
        x_k - p_{m-1} - (p_m - p_{m-1}) \frac{\tau_k - t_{m-1}}{t_m - t_{m-1}}
    \Bigr\|^2
    \,, $$

for $
I_m = \{k\colon \tau_k \in [t_{m-1}, t_m)\}
$, $
    p = (p_m)_{m=0}^M \in \mathbb{R}^d
$, and $
    t = (t_m)_{m=0}^M \uparrow
$ with $
    0 = t_0 < \cdots < t_M = 1
$ and $p_0 = 0$.

In [ ]:
pass

<br>

### DP: Indices and points (unfinished)

Instead of figuring out the segment times $t_m\in [0, 1]$, $t_m\uparrow$,
let's just fit a bunch of sequentially connected segments. And slightly
disregard continuity requirement.

Also we have $k_0 = 0$ and $k_M = K$.

Let $V(M, K)$ denote the best fit of $M$ segments on the data up to $K$:

$$
\begin{align}
V_M(x, l)
    &= \min_{p, k}
        \sum_{i=0}^{k-1}
            \Bigl\| x_i - p \frac{i}{k} \Bigr\|^2
        + V_{M-1}(x_{k:} - p, l-k)
    \,, \\
V_1(x, k)
    &= \min_{p}
        \sum_{i=0}^{k-1}
            \Bigl\| x_i - p \frac{i}{k} \Bigr\|^2
    \,,
\end{align} $$
where $x_{j:} = (x_k)_{k\geq j}$.

If $\tau = (t)_{t=0}^{T-1}$, then $
    p = (\tau^\top \tau)^{-1} \tau^\top X
$ for $X = (x_t)_{t=0}^{T-1}$. The fit is $
    \mathop{tr} \bigl(I-\tau (\tau^\top \tau)^{-1} \tau^\top\bigr) X X^\top
$.

$$
p = \frac{6k}{(k-1) (2k-1)}\sum_{i=0}^{k-1} \frac{i}{k} x_i
\,. $$

<br>

### DP: Discontinuous local regression

Essentially we need to cluster the points using the fact that the mean in each
cluster is in fact a linear segment.

$$
V(M, K)
    = \min_{0 < j < K} V(M-1, j)
        + \min_{p_0, p_1} \sum_{i=j}^{K-1} \Bigl\|
            x_i - p_0 - (p_1 - p_0) \frac{i-j}{K-j}
        \Bigr\|^2
    \,. $$

Fit a single segment

$$
\| Y - 1\beta_0 - X \beta \|^2
    \,. $$
$$
1^\top Y - 1^\top 1 \beta_0 - 1^\top X \beta = 0
    \Leftrightarrow
    \beta_0 = (1^\top 1)^{-1} 1^\top (Y - X \beta)
    \,. $$

$$
\| (I - 1(1^\top 1)^{-1} 1^\top) Y - (I - 1(1^\top 1)^{-1} 1^\top) X \beta \|^2
    \,. $$

In [ ]:
n_min_pts = 3

def segment_fit_py(P, i, j, n_min_pts=n_min_pts):
    if n_min_pts > j - i:
        return np.inf

    Y = P[i:j]
    Y = Y - Y.mean(0)

    t = np.arange(-(j - i - 1) / 2, (j - i + 1) / 2)
    resid = Y - t.reshape(-1, 1) * np.dot(t, Y) / np.dot(t, t)
    return 0.5 * (resid * resid).sum()

In [ ]:
import numba as nb

@nb.jit(["f8(f8[:, :], i8, i8, i8)"], nopython=True)
def segment_fit_py(P, i, j, n_min_pts):
    if n_min_pts > j - i:
        return np.inf

    Y, mean = P[i:j], np.zeros_like(P[:1])
    for k in range(i, j):
        mean += P[k]
    Y = Y - mean / (j-i)

    t = np.arange(-(j - i - 1) / 2, (j - i + 1) / 2)
    resid = Y - t.reshape(-1, 1) * np.dot(t, Y) / np.dot(t, t)
    return 0.5 * (resid * resid).sum()

Given points $
    P \in \mathbb{R}^{K\times d}
$ and $
    \theta = (\tfrac{k}{K-1})_{k=0}^{K-1}
$ minimize $
    \| P - (1-\theta)p_0 - \theta p_1 \|^2
$ with respect to $p_0$ and $p_1$.

In [ ]:
V = np.full((M, len(P)), np.inf)
for k in range(1, len(P)):
    V[0, k] = segment_fit_py(P, 0, k, n_min_pts)

for m in tqdm.tqdm(range(1, M)):
    for k in range(1, len(P)):
        fits = [segment_fit_py(P, j, k, n_min_pts) for j in range(k)]
        V[m, k] = min(V[m - 1, :k] + np.r_[fits])

Return the endpoints of a fit segment.

In [ ]:
def segment_get_py(P, i, j, n_min_pts=n_min_pts):
    Y, t = P[i:j], np.arange(-(j - i - 1) / 2, (j - i + 1) / 2)
    mY = Y.mean(axis=0, keepdims=True)

    beta = np.dot(t, Y - mY) / np.dot(t, t)
    return mY + t[0] * beta, mY + t[-1] * beta

Reconstruct by backtracking through the value function of the DP.

In [ ]:
def reconstruct(P, V):
    m, k = M-1, len(P)
    while m > 0:
        yield k
        fits = [segment_fit_py(P, j, k, n_min_pts) for j in range(k)]
        k = (np.r_[fits] + V[m-1, :k]).argmin()
        m -= 1
    yield k
    yield 0

Project the data onto segments

In [ ]:
def project(P, V):
    ix = [*reconstruct(P, V)][::-1]

    out = np.full_like(P, np.nan)
    for i, j in zip(ix, ix[1:]):
        p0, p1 = segment_get_py(P, i, j, n_min_pts)
        t = np.c_[np.linspace(0, 1, j-i)]
        out[i:j] = p0 * (1 - t) + p1 * t
    return out

In [ ]:
from itertools import chain

def project_with_averaging(P, V):
    ix = [*reconstruct(P, V)][::-1]
    
    # add zero and final endpoints and average in pairs
    points = [segment_get_py(P, i, j, n_min_pts) for i, j in zip(ix, ix[1:])]
    points = [np.zeros_like(P[:1])] + [*chain(*points)] + [points[-1][-1]]
    points = [0.5*(a+b) for a, b in zip(points[0::2], points[1::2])]
    
    px = [*zip(ix, points)]

    out = np.full_like(P, np.nan)
    for (i, p0), (j, p1) in zip(px, px[1:]):
        t = np.c_[np.linspace(0, 1, j-i)]
        out[i:j] = p0 * (1 - t) + p1 * t
    return out

Reconstruct

In [ ]:
# out = project(P, V)
out = project_with_averaging(P, V)

Fit error

In [ ]:
error = (track_true.to_numpy() - out).ravel()

np.sqrt((error * error).mean())

Plot

In [ ]:
colors = plt.cm.rainbow(np.linspace(0, 1, M))
fig, ax = plt.subplots(1, 1, figsize=(16, 9))

ix = [*reconstruct(P, V)][::-1]
for m, (i, j) in enumerate(zip(ix, ix[1:])):
    ax.scatter(*P[i:j].T, c=colors[[m]])

# for i, j in zip(ix, ix[1:]):
#     p0, p1 = segment_get_py(P, i, j)
#     ax.plot(*np.r_[p0, p1].T, c="k", lw=3)

ax.plot(track_true.X, track_true.Y, c="k", zorder=10, lw=5)
ax.plot(out[:, 0], out[:, 1], c="fuchsia", lw=3, zorder=10)

<br>

## Greedy Aglomerative Clustering Approach

This is a greedy approximation to `Indices and points`.

Collect the points into $G$ contiguous groups of some minimal length (`n_perseg`),
then fit consecutively fit each $p_g$ assuming $p_0 = 0$, $k_0 = 0$, and $k_G = K$:

$$
V([k_{g-1}, k_g), p_{g-1})
    = \min_{p_g} \sum_{k=k_{g-1}}^{k_g - 1} \Bigl\|
        (x_k - p_{g-1})
        - (p_g - p_{g-1}) \frac{k - k_{g-1}}{k_g - k_{g-1} - 1}
    \Bigr \|^2
    \,. $$

Then from the constructed sequence $(p_g)_{g=0}^G$ compute the directional vectors
$u_g = \tfrac{p_g - p_{g-1}}{\|p_g - p_{g-1}\|}$. Then find the most collinear
directions: $g_* = \arg \max_{g=1}^{G-1} u_g^\top u_{g+1}$. Proceed by merging
group $[k_{g_*-1}, k_{g_*})$ with $[k_{g_*}, k_{g_*+1})$. Finally, reestimate
$V([k_{g_*-1}, k_{g_*+1}), p_{g_*-1})$ and then **all** $
    V([k_{g-1}, k_g), p_{g-1})
$ for $g\geq g_* + 1$ over **new intervals** from the new point $p_{g_*}$.

Seems like the worst complexity is cubic.

In [ ]:
def refit_splits(data, p, u, i0, *splits, c=-np.inf):
    # Observe that in a continuous piecewise linear curve, the last
    #  endpoint of each segment is the first endpoint of the next
    # segment. Thus take that `p` is an estimate of the endpoint
    #  associated with data[i0].

    yield p, u, i0, -np.inf
    d = np.zeros_like(data[:1])
    for i1 in splits:
        if i1 == i0:
            continue

        # fit regression x - p \sim \tau d w.r.t `d` \tau \in [0, 1]
        x = data[i0:i1 + 1] - p  # don't care if we fall off the right end of data
        if len(x) >= 2:
            t = np.linspace(0, 1, len(x))
            d = np.dot(t, x) / np.dot(t, t)

        # normalize the direction
        v = d / np.linalg.norm(d)
        c = (-np.inf if u is None else u @ v)
        yield p + d, v, i1, c

        i0, p, u = i1, p + d, v

`n_perseg` is very important.

In [ ]:
n_perseg = max(int(np.sqrt(len(P))), 2)

Coalesce clusters from ground up.

In [ ]:
n_fullseg = (len(P) - 1) // n_perseg
splits = [*range(0, len(P) - n_perseg + 1, n_perseg), len(P)]

segments = [*refit_splits(P, np.zeros_like(P[0]), None, *splits)]
levels = [segments[:]]
while len(segments) > 2:
    k = np.argmax([c for p, u, i, c in segments]) - 1
    p, u, i0, c = segments[k-1]
    del splits[k]

    _, *tail = refit_splits(P, p, u, i0, *splits[k:])
    segments = segments[:k] + tail

    levels.append(segments[:])

Reconstruct

In [ ]:
pts = [p for p, u, i, c in levels[-M]]
ixs = [i for p, u, i, c in levels[-M]]

out = np.full_like(P, np.nan)
ptix = [*zip(ixs, pts)]
for (i, pi), (j, pj) in zip(ptix, ptix[1:]):
    t = np.c_[np.linspace(0, 1, j-i)]
    out[i:j] = pi * (1 - t) + pj * t

Fit error

In [ ]:
error = (track_true.to_numpy() - out).ravel()
np.sqrt((error * error).mean())

In [ ]:
colors = plt.cm.rainbow(np.linspace(0, 1, len(ixs)-1))
fig, ax = plt.subplots(1, 1, figsize=(16, 9))

for m, (i, j) in enumerate(zip(ixs, ixs[1:])):
    ax.scatter(*P[i:j].T, c=colors[[m]])

ax.plot(track_true.X, track_true.Y, c="k", zorder=10, lw=5)
ax.plot(out[:, 0], out[:, 1], c="fuchsia", lw=3, zorder=10)

<br>

## Markov chain functional demux

Use mixture separation but assume that latent variables $z_k\in \{1 .. M\}$
follow a $1$-st order markov chain with $
    \pi_{ij} = 0
$ if $i < j$ or $i > j + 1$, where $
    \mathbb{P}(z_{k+1} = i \mid z_k = j) = \pi_{ij}
$ (lower bidiagonal matrix):
$$
\pi = \begin{pmatrix}
    \pi_{11} & 0 & 0 & \cdots & 0 \\
    \pi_{21} & \pi_{22} & 0 & \cdots & 0 \\
    0 & \pi_{32} & \pi_{33} & \cdots & 0 \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    0 & 0 & 0 & \cdots & \pi_{MM}
\end{pmatrix}
\,. $$


Then at the E-step we estimate the current states (filtering + smoothing):
$$
q_{ki} = \sum_{z_{-k}}
    \mathbb{P}(z_k = i \mid z_{-k}, x, p)
    \mathbb{P}(z_{-k} \mid x, p)
\,, $$
(verify)

<br>

In [ ]:
X = np.diff(P, axis=0)

In [ ]:
plt.scatter(*X.T, c=plt.cm.rainbow(np.linspace(0, 1, len(X))))

In [ ]:
mu = np.random.randn(M, *X.shape[1:])
sigma2 = 25.0**2


In [ ]:
pr = 0.99
Pi = pr * np.eye(M) + (1-pr) * np.eye(M, k=1)

Pi[-1, -1] = 1.

In [ ]:
Pi[3, 3] = 0.5 ; Pi[3, 4] = 0.5
Pi[5, 5] = 0.5 ; Pi[5, 6] = 0.5

In [ ]:
Pi

See notes under the kb

Let $
    \xi_{t\mid T} = \Pr\bigl(
        S_t \mid x_T \cdots x_1
    \bigr)
$ and we assume that $
    S_t \bot (X_\tau)_{\tau < t} \mid S_{t-1}
$ as well as $
    S_t \bot (X_\tau)_{\tau > t} \mid S_{t+1}
$. Then for $t > T$ we have $
    \xi_{t\mid T} = \Pi^\top \xi_{t-1\mid T}
$, $
    \xi_{t\mid t} = \eta_t \odot \xi_{t\mid t-1}
$ and $
    \xi_{t\mid T} = \xi_{t\mid t}
        \odot \Pi (\xi_{t+1\mid T} \div \xi_{t+1\mid t})
$ for $t < T$.

$$
(2\pi)^{-\tfrac{d}2} \lvert\Sigma \rvert^{-\tfrac12}
    \exp\{
        -\tfrac12 (x-\mu)^\top \Sigma^{-1} (x-\mu)
    \}
\,. $$
For $\Sigma = \sigma^2 I$ we have $
    \lvert\Sigma \rvert = (\sigma^2)^d
$.

In [ ]:
def pdf(x, mu, sigma2):
    return np.exp(-0.5 * len(x) * (
        np.log(2 * np.pi * sigma2)
        + (x - mu) @ (x - mu) / (sigma2 * len(x))
    ))

In [ ]:
# e-step
xit = np.zeros((len(X), M))

# filtering: \xi_{t \mid t}
xit[0, 0] = 1.  # start at segment 0
for t in range(1, len(X)):
    probs = np.r_[[pdf(X[t], m, sigma2) for m in mu]]
    xit[t] = probs * (Pi.T @ xit[t-1])
    xit[t] /= xit[t].sum()

# smoothing: \xi_{t\mid T}
xiT = np.zeros((len(X), M))
xiT[-1, -1] = 1.  # end at semgent M-1  # xiT[-1] = xit[-1]
for t in range(len(X)-1, 0, -1):
    ratio = np.nan_to_num(xiT[t] / (Pi.T @ xit[t-1]))
    xiT[t-1] = xit[t-1] * (Pi @ ratio)

# m-step
wx = X[:, np.newaxis] * xiT[..., np.newaxis]
mu = wx.sum(0) / xiT.sum(0)[:, np.newaxis]

fig, ax = plt.subplots(1, 2)
ax[0].plot(xiT);
ax[1].scatter(*mu.T, c=plt.cm.rainbow(np.linspace(0, 1, num=len(mu))));

In [ ]:
plt.scatter(*X.T, c=plt.cm.rainbow(np.linspace(0, 1, len(X))))

plt.scatter(*mu.T, c="k", s=50)

In [ ]:
plt.plot(xit);

In [ ]:
assert False

<br>